## reconstructing molecular conformations from pairwise distances

### setup

In [ ]:
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sys
import math
import time
import pickle
from copy import deepcopy

import numpy as np
import pandas as pd

import sklearn.preprocessing as pre
from sklearn.model_selection import train_test_split
from sklearn import manifold

from scipy.linalg import orthogonal_procrustes

from collections import OrderedDict

import mdtraj as md
import nglview as nv

In [ ]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2'''

In [ ]:
# fix random seed for reproducibility
randstate = 200184
np.random.seed(randstate)

### loading trajectory

In [ ]:
trj_dir = "../C24/"
pdb_file = os.path.join(trj_dir, "after_pr_protein.pdb")
trj_file = os.path.join(trj_dir, "md_protein_fit.xtc")

In [ ]:
# loading
traj = md.load(trj_file, top=pdb_file)

In [ ]:
# rot+trans alignment of trajectory to first frame
traj.center_coordinates(mass_weighted=False)
traj.superpose(traj[0])

In [ ]:
print('How many frames? %s' % traj.n_frames)
print('How many atoms?    %s' % traj.n_atoms)
print(traj.xyz.shape)

In [ ]:
# visualizing
view = nv.show_mdtraj(traj)
view.remove_cartoon()
view.add_ball_and_stick()
view

### loading pairwise distances

In [ ]:
#We need the pairwise distance matrix predicted by our artificial neural network code 5
npzfile = np.load('../5_ANN_dMapaa2pd/ANN_pred.npz')
sorted(npzfile.files)
pd = npzfile['pd_pred']

In [ ]:
# no. of frames
nFrames = pd.shape[0]
print('nFrames = %d' % nFrames)

In [ ]:
# no. of atoms (between which pairwise distances are computed) and no. of pairs
Npairs = pd.shape[1]
print('Npairs = %d' % Npairs)

N = (1. + np.sqrt(1.+8.*Npairs))/2.
N = np.int(N)
print('N = %d' % N)

assert N*(N-1) == 2*Npairs

In [ ]:
assert traj.n_frames == nFrames
assert traj.n_atoms == N

### reconstructing conformations from pairwise distances by cMDS

cMDS permits reconstruction up to reflection

OUTPUTS:

traj_o_init.pdb, traj_o.xtc -- (subsampled) original trajectory

traj_r_init.pdb, traj_r.xtc -- reconstruction of traj_o with conformations fitted wrt reference conformation

traj_r_oFit_init.pdb, traj_r_oFit.xtc -- reconstruction of traj_o with conformations fitted wrt corresponding true conformation (unrealistic but avoids any spurious mirror inversions induced by choice of reference conformation)

In [ ]:
# reconstruction parameters for C24 trajectory in paper (cMDS is expensive)
startFrame = 100000
stopFrame = 500000
strideFrame = 1000 # Parameters for the C24 test data
#startFrame = 0
#stopFrame = 100000
#strideFrame = 1000 # Paramaters for the C24 training data

In [ ]:
# establishing cMDS object
mds = manifold.MDS(n_components=3, max_iter=3000, eps=1e-9, random_state=200184, dissimilarity="precomputed", n_jobs=1)

In [ ]:
# initializing mdtraj containers
# N.B. traj_r_oFit is not realistic in practice since all true frames will not be available but we do it to 
#      avoid the possibility of spurious mirror inversions induced by reference configuration since cMDS specifies 
#      conformation only up to arbitrary mirror inversion

traj_idx = np.arange(startFrame,stopFrame,strideFrame) # frame indices
traj_o = deepcopy(traj[startFrame:stopFrame:strideFrame])  # true trajectory frames which we will reconstruct by pairwise distances
traj_r = deepcopy(traj_o) # reconstruction of true trajectory frames aligned to reference structure (will be overwritten by reconstructions)
traj_r_oFit = deepcopy(traj_o) # reconstruction of true trajectory frames aligned to the corresponding true frame (will be overwritten by reconstructions)

In [ ]:
# performing reconstruction
print('Commencing frame reconstruction...')

i=0
for frame in range(startFrame,stopFrame,strideFrame):

    D_r = np.zeros((N, N))
    D_r[np.triu_indices(N,1)] = pd[frame,:]
    D_r += D_r.T

    pos_r = mds.fit(D_r).embedding_
    pos_r = pos_r - np.mean(pos_r,axis=0)
    
    # traj_r: orthogonal Procustes (rot+trans+inversion, no scaling) wrt ref conformation
    pos_ref = deepcopy(traj.xyz[0] - np.mean(traj.xyz[0],axis=0))
    Rt, _ = orthogonal_procrustes(pos_r, pos_ref)
    #if np.linalg.det(Rt) < 0:
    #    print('\t->Reflection detected at frame index %d' % (frame))
    pos_r_align = pos_r.dot(Rt)
    traj_r.xyz[i] = deepcopy(pos_r_align)
    
    # traj_r_oFit: orthogonal Procustes (rot+trans+inversion, no scaling) wrt ref conformation
    pos_ref = deepcopy(traj_o.xyz[i] - np.mean(traj_o.xyz[i],axis=0))
    Rt, _ = orthogonal_procrustes(pos_r, pos_ref)
    #if np.linalg.det(Rt) < 0:
    #    print('\t->Reflection detected at frame index %d' % (frame))
    pos_r_align = pos_r.dot(Rt)
    traj_r_oFit.xyz[i] = deepcopy(pos_r_align)
    
    i+=1
    
    if np.mod(i,10) == 0:
        print('\tReconstructed %d/%d frames' % (i,np.floor((stopFrame-startFrame)/strideFrame)))

print('DONE!')

In [ ]:
# viewing original trajectory
view = nv.show_mdtraj(traj_o)
view.remove_cartoon()
view.add_ball_and_stick()
view

In [ ]:
# viewing reconstructed trajectory
view = nv.show_mdtraj(traj_r)
view.remove_cartoon()
view.add_ball_and_stick()
view

In [ ]:
# viewing reconstructed trajectory aligned to corresponding true conformation
view = nv.show_mdtraj(traj_r_oFit)
view.remove_cartoon()
view.add_ball_and_stick()
view

### computing RMSD between true and reconstructed trajectories

In [ ]:
RMSD_r = np.sqrt(np.mean((traj_o.xyz - traj_r.xyz)**2))
print('RMSD_r = %.2e (nm)' % RMSD_r)

RMSD_r_oFit = np.sqrt(np.mean((traj_o.xyz - traj_r_oFit.xyz)**2))
print('RMSD_r_oFit = %.2e (nm)' % RMSD_r_oFit)

### saving trajectories

In [ ]:
traj_o[0].save_pdb('traj_o_init.pdb')
traj_o.save_xtc('traj_o.xtc')

traj_r[0].save_pdb('traj_r_init.pdb')
traj_r.save_xtc('traj_r.xtc')

traj_r_oFit[0].save_pdb('traj_r_oFit_init.pdb')
traj_r_oFit.save_xtc('traj_r_oFit.xtc')